- Default PSO parameters
- max_back=20

In [1]:
from joblib import Parallel, delayed
import multiprocessing
import random
import pickle

import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import pygmo as pg

import tengp
import symreg
from experiment_settings import nguyen7_funset, pagie_funset, keijzer_funset, korns12_funset, vlad_funset

In [9]:
TRIALS = 100
OUTPUT_FOLDER  = 'results/pso_default_20_logs/'
PARALLEL = False

In [6]:
def run_parallel(current_data):
    if not PARALLEL:
        logs = run_experiment(current_data, cost_function)
    else:
        name, (x_train, y_train, x_test, y_test), params = current_data
        print(name)

        bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]

        logs = Parallel(n_jobs=num_cores)(delayed(run_experiment_instance)(_, cost_function, x_train, y_train, params, bounds)
                                            for _ in range(TRIALS))
    return logs

def run_experiment(data_item, cost_function):
    logs = []
    
    name, (x_train, y_train, x_test, y_test), params = data_item
    print(name)
    
    bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]
    
    for i in range(TRIALS):
        log = run_experiment_instance(i, cost_function, x_train, y_train, params, bounds)
        logs.append(log)
    
    return logs


def run_experiment_instance(i, cost_function, x_train, y_train, params, bounds):
    print(i, end=',')
    prob = pg.problem(cost_function(np.c_[np.ones(len(x_train)), x_train], y_train, params, bounds))
    algo = pg.algorithm(pg.pso(gen=2000))
    algo.set_verbosity(1)
    pop = pg.population(prob, 50)
    pop = algo.evolve(pop)
    uda = algo.extract(pg.pso)
        
    return [x[2] for x in uda.get_log()]

class cost_function:
    def __init__(self, X, Y, params, bounds):
        self.params = params
        self.bounds = bounds
        self.X = X
        self.Y = Y
    
    def fitness(self, x):      
        individual = tengp.individual.NPIndividual(list(x), self.bounds, self.params)
        
        pred = individual.transform(self.X)
        
        try:
            return [mean_squared_error(pred, self.Y)]
        except ValueError:
            return [10000000000]
        
    def get_bounds(self):
        return self.bounds

In [7]:
kw_params = {'real_valued': True, 'max_back': 20}

params_nguyen4 = tengp.Parameters(2, 1, 1, 50, nguyen7_funset, **kw_params)
params_nguyen7 = tengp.Parameters(2, 1, 1, 50, nguyen7_funset, **kw_params)
params_nguyen10 = tengp.Parameters(3, 1, 1, 50, nguyen7_funset, **kw_params)
params_pagie1  = tengp.Parameters(3, 1, 1, 50, pagie_funset, **kw_params)
params_keijzer6 = tengp.Parameters(2, 1, 1, 50, keijzer_funset, **kw_params)
params_korns = tengp.Parameters(6, 1, 1, 50, korns12_funset, **kw_params)
params_vlad = tengp.Parameters(6, 1, 1, 50, vlad_funset, **kw_params)

all_params = [params_nguyen7, params_pagie1, params_keijzer6, params_korns, params_vlad]

data = [
    ('nguyen4', symreg.get_benchmark_poly(random, 6), params_nguyen4),
    ('nguyen7', symreg.get_benchmark_nguyen7(random, None), params_nguyen7),
    ('nguyen10', symreg.get_benchmark_nguyen10(random, None), params_nguyen10),
    ('pagie1', symreg.get_benchmark_pagie1(random, None), params_pagie1),
    ('keijzer6', symreg.get_benchmark_keijzer(random, 6), params_keijzer6),
    ('korns12', symreg.get_benchmark_korns(random, 12), params_korns),
    ('vladislasleva4', symreg.get_benchmark_vladislasleva4(random, None), params_vlad)
]

num_cores = multiprocessing.cpu_count() - 1

# Nguyen 4

In [ ]:
%%time

pg.set_global_rng_seed(seed = 42)
logs = run_parallel(data[0])
pickle.dump(logs, open(f'{OUTPUT_FOLDER}ng4_log', 'wb'))

# Nguyen 7


In [ ]:
%%time

pg.set_global_rng_seed(seed = 42)
logs = run_parallel(data[1])
pickle.dump(logs, open(f'{OUTPUT_FOLDER}ng7_log', 'wb'))

# Pagie

In [10]:
%%time

pg.set_global_rng_seed(seed = 42)
logs = run_parallel(data[3])
pickle.dump(logs, open(f'{OUTPUT_FOLDER}pag1_log', 'wb'))

pagie1
0,

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:106: RuntimeWarning: invalid value encountered in power
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:107: RuntimeWarning: invalid value encountered in power
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:106: RuntimeWarning: overflow encountered in power
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:11: RuntimeWarning: invalid value encountered in greater
  return np.log(x, out=np.copy(x), where=x>0)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:107: RuntimeWarning: overflow encountered in power
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:108: RuntimeWarning: invalid value encountered in add
  return l + u
/mnt/c/Use

1,

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:106: RuntimeWarning: invalid value encountered in multiply
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:107: RuntimeWarning: invalid value encountered in add
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:107: RuntimeWarning: divide by zero encountered in power
  u = a*f_u(L, U)
/home/jarino/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


2,3,4,5,6,7,8,9,10,

...

11,12,13,14,15,16,17,18,19,20,21,

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:107: RuntimeWarning: overflow encountered in add
  u = a*f_u(L, U)


22,23,24,25,26,27,28,29,30,31,32,33,34,

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:141: RuntimeWarning: overflow encountered in add
  value = (1-coeff)*self.nodes[lower].value + coeff*self.nodes[upper].value


35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,CPU times: user 3h 28min 45s, sys: 16.2 s, total: 3h 29min 1s
Wall time: 3h 33min 45s


# Keijzer 6

In [ ]:
%%time

pg.set_global_rng_seed(seed = 42)
logs = run_parallel(data[4])
pickle.dump(logs, open(f'{OUTPUT_FOLDER}kei6_log', 'wb'))

# Korns 12

In [8]:
%%time

pg.set_global_rng_seed(seed = 42)
logs = run_parallel(data[5])
pickle.dump(logs, open(f'{OUTPUT_FOLDER}korns12_log', 'wb'))

korns12


/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:38: RuntimeWarning: overflow encountered in exp
  return np.exp(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:38: RuntimeWarning: overflow encountered in exp
  return np.exp(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:11: RuntimeWarning: invalid value encountered in greater
  return np.log(x, out=np.copy(x), where=x>0)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimizatio

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:106: RuntimeWarning: invalid value encountered in add
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:141: RuntimeWarning: invalid value encountered in add
  value = (1-coeff)*self.nodes[lower].value + coeff*self.nodes[upper].value
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:106: RuntimeWarning: invalid value encountered in add
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:104: RuntimeWarning: invalid value encountered in multiply
  L = (1-b)*x_l1 + b*x_u1
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:104: RuntimeWarning: invalid value encountered in multiply
  L = (1-b)*x_l1 + b*x_u1
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:107: RuntimeWarning: invalid value encountered in multiply
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:107: RuntimeWarning: invalid value encountered in multiply
  u = a*f_u(L

CPU times: user 8.27 s, sys: 4.19 s, total: 12.5 s
Wall time: 16h 50min 17s


# Vladislasleva 4

In [ ]:
%%time

pg.set_global_rng_seed(seed = 42)
logs = run_parallel(data[6])
pickle.dump(logs, open(f'{OUTPUT_FOLDER}vlad4_log', 'wb'))